# Slowly Varying Regression under Sparsity

This notebook contains a demo run for some of the algorithms described in the paper Slowly Varying Regression under Sparsity.

We implement all algorithms in Julia programming language (version 1.6) and using the JuMP.jl modeling language for mathematical optimization (version 0.21). We solve the optimization models using the Gurobi commercial solver (version 9.5).

## 1. Import modules

In [ ]:
using JLD2

include("src/svar_cutplane.jl")
include("src/svar_heuristic.jl")
include("src/regression_sum_of_norms.jl")
include("src/sparse_regression.jl")
include("src/cv.jl")
include("src/utils.jl")
include("src/svar_visualization.jl")
;

## 2. Load data

In [ ]:
###
# Select dataset
###

datapath = "data/energy_hour.jld2"
# datapath = "data/housing.jld2"
X, Y, edges = load(datapath, "X"), load(datapath, "Y"), load(datapath, "edges")


###
# Train-valid-test split
###

N_total,T,D = size(X)
train_size = convert(Int, floor(N_total*0.6))
valid_size = convert(Int, floor(N_total*0.2))
test_size = convert(Int, floor(N_total*0.2))
all_indices = collect(1:N_total)
train_indices = sample(all_indices, train_size, replace=false)
all_indinces = setdiff(all_indices, train_indices)
valid_indices = sample(all_indices, valid_size, replace=false)
all_indinces = setdiff(all_indices, valid_indices)
test_indices = sample(all_indices, test_size, replace=false)

X_train, Y_train = view(X,train_indices,:,:), view(Y,train_indices,:)
X_valid, Y_valid = view(X,valid_indices,:,:), view(Y,valid_indices,:)
X_test, Y_test = view(X,test_indices,:,:), view(Y,test_indices,:)

X_train, Y_train, X_valid, Y_valid, X_test, Y_test = standardize_data(X_train, Y_train, X_valid, Y_valid, X_test, Y_test)
;

## 3. Cross validation using svar_heuristic

In [ ]:
###
# Define search and constant hyperparameters
###

search_params = Dict(
        :lambda_reg => 3,
        :lambda_svar => 3,
        # ENERGY
        :sparsity => [10,15,20],
        :global_sparsity_relative => [3,5],
        :sparsely_varying => [20,30],
#         # HOUSING
#         :sparsity => [30,40,50,60],
#         :global_sparsity_relative => [10,30],
#         :sparsely_varying => [50,100],
    )

const_params = Dict(
            :time_limit => 30.,
            :verbose => 0,
            :decrease_final_regularizer => true
        )

global DBG = stdout # print debug statement in stdout
global DBG_list = [:progress] # only show progress debug statemets (e.g., hide memory related ones)

###
# Run cross validation
###

grid_search = holdout_grid_search(
    X_train,
    Y_train,
    svar_heuristic,
    search_params,
    const_params,
    X_valid=X_valid,
    Y_valid=Y_valid,
)


In [ ]:
###
# Extract refit model and selected hyperparameter values
###

lnr_heuristic = grid_search.lnr_refit

println("Heuristic sparsity params: $(eval_sparsity(lnr_heuristic.z, edges))")
println("Heuristic test r2: $(eval_r2(X_test,Y_test,lnr_heuristic.beta))")

sparsity = lnr_heuristic.lnr_params[:sparsity]
global_sparsity = lnr_heuristic.lnr_params[:global_sparsity]
sparsely_varying = lnr_heuristic.lnr_params[:sparsely_varying]
lambda_svar = lnr_heuristic.lnr_params[:lambda_svar]
;

## 4. Run svar_cutplane

In [ ]:
###
# Run cutplane
###

lnr_cutplane = svar_cutplane(
    vcat(X_train, X_valid), 
    vcat(Y_train, Y_valid),
    edges=edges,
    sparsity=sparsity,
    global_sparsity=global_sparsity,
    sparsely_varying=sparsely_varying,
    lambda_svar=lambda_svar,
    decrease_final_regularizer=true,
    time_limit=300.,
    verbose=1
)

println("Cutplane sparsity params: $(eval_sparsity(lnr_heuristic.z, edges))")
println("Cutplane test r2: $(eval_r2(X_test,Y_test,lnr_heuristic.beta))")

## 5. Visualize svar_cutplane 

In [ ]:
coef_df = create_coef_df(
    lnr_cutplane.beta,
)

g = plot_feature_variation(
    coef_df,
    edges,
    fontsize=3,
    method=:circular
)

display(g)